In [ ]:
import cirq
import numpy as np

# Function to create an oracle gate from a given unitary matrix
def create_oracle_gate(oracle_matrix):
    return cirq.MatrixGate(oracle_matrix)

# Input a function definition instead of unitary matrix values
def input_function_definition():
    print("Paste a Python function named f(x0, x1) that returns 0 or 1.")
    print("End with a single line: END")
    print("\nExample:\n"
          "def f(x0, x1):\n"
          "    return x0 ^ x1\n"
          "END\n")

    lines = []
    while True:
        line = input()
        if line.strip() == "END":
            break
        lines.append(line)

    code = "\n".join(lines)

    env = {}
    exec(code, env, env)  # loads f into env

    if "f" not in env or not callable(env["f"]):
        raise ValueError("You must define a function named f(x0, x1).")

    return env["f"]

# Build the oracle matrix from the function f
def build_oracle_matrix_from_f(f):
    """
    We have 3 qubits total: q0, q1 are input bits; q2 is ancilla y.
    Oracle definition: Uf |x0, x1, y> = |x0, x1, y XOR f(x0, x1)>

    IMPORTANT (Cirq ordering):
    For qubits = [q0, q1, q2], Cirq uses little-endian indexing:
      index = x0*1 + x1*2 + y*4
    """
    U = np.zeros((8, 8), dtype=np.complex128)

    for x0 in (0, 1):
        for x1 in (0, 1):
            fx = f(x0, x1)
            if fx not in (0, 1):
                raise ValueError(f"f({x0}, {x1}) must return 0 or 1, got {fx}")

            for y in (0, 1):
                in_index = x0 + 2 * x1 + 4 * y
                y_out = y ^ fx
                out_index = x0 + 2 * x1 + 4 * y_out

                # This creates a permutation matrix (always unitary)
                U[out_index, in_index] = 1.0

    return U

def determine_function_type(circuit_result):
    # Get the measurement outcomes of the initial two qubits
    measurement_outcomes = [circuit_result.measurements[f'result_{i}'][0][0] for i in range(2)]

    # Determine the type of function based on the measurement outcomes
    if all(outcome == 0 for outcome in measurement_outcomes):
        return "constant"
    else:
        return "balanced"

# Define Deutsch's algorithm (your circuit structure stays the same)
def deutsch_algorithm(oracle_gate):
    # Define qubits
    qubits = cirq.LineQubit.range(3)

    # Create a quantum circuit
    circuit = cirq.Circuit()

    # Apply X gate (NOT gate) on the third qubit
    circuit.append(cirq.X(qubits[-1]))

    # Apply Hadamard gate to all qubits
    circuit.append([cirq.H(q) for q in qubits])

    # Apply the matrix gate on all inputs
    circuit.append(oracle_gate.on(*qubits))  # safer than oracle_gate(*qubits)

    # Apply Hadamard gate to all outputs except the last qubit
    for qubit in qubits[:-1]:
        circuit.append(cirq.H(qubit))

    # Measure the initial two qubits
    circuit.append([cirq.measure(q, key=f'result_{i}') for i, q in enumerate(qubits[:2])])

    return circuit


# Instead of prompting user for 64 values, prompt for function f
f = input_function_definition()

# Auto-build the oracle matrix from f
oracle_matrix = build_oracle_matrix_from_f(f)

# Create the oracle gate from the auto-built unitary matrix
oracle_gate = create_oracle_gate(oracle_matrix)

# Run Deutsch's algorithm with the oracle gate
circuit = deutsch_algorithm(oracle_gate)

# Simulate the circuit 10 times and collect results
simulator = cirq.Simulator()
measurement_outcomes_list = []

for _ in range(10):
    result = simulator.run(circuit)
    measurement_outcomes = [result.measurements[f'result_{i}'][0][0] for i in range(2)]
    measurement_outcomes_list.append(measurement_outcomes)

# Print all measurement outcomes
print("Measurement outcomes:", measurement_outcomes_list)

# Determine the type of function based on the measurement outcomes
function_type = determine_function_type(result)
print("Function type:", function_type)

# Print the circuit
print("\nCircuit:")
print(circuit)


Paste a Python function named f(x0, x1) that returns 0 or 1.
End with a single line: END

Example:
def f(x0, x1):
    return x0 ^ x1
END

